In [1]:
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(12,GPIO.OUT)
GPIO.output(12,GPIO.HIGH)

/tmp/ipykernel_2020/3736153917.py:3: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(12,GPIO.OUT)


In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import pyrealsense2 as rs
import serial
import numpy as np
import time
from threading import Thread
import threading
import csv
import DataDeal265 as dd
import struct
import queue
import torch
import cv2
import numpy as np
import ipywidgets
import pyzbar.pyzbar as pyzbar
import math
from cnn import *

cnn=Net()#加载网络
# 加载网络模型
cnn.load_state_dict(torch.load('weight/0.9801-net_params.pt',map_location='cpu'))
CopterTakingOff =1
TargetPosition = [0.0,0.0,0.0]

#图像大小
cap_x=360
cap_y=240
# cp1=[0.0,0.0]#起点
# cp2=[-1.0,0.0]#数字识别点一
# # cp3=[-1,1]#二维码识别点
# X坐标 Y坐标
cp0=[0.0,0.0]#起点
#左边路径
# cp1=[0.0,1.0]#数字识别点一
# cp2=[-1.0,1.0]
cp1=[-1.7,1.6]#数字识别点一
cp2=[-1.7,3.5]#数字识别点二
cp3=[-1.7,5.5]#数字识别点三

cp4=[-0.2,4.9]#二维码识别点
#右边路径
cp5=[1.45,5.7]#数字识别点四
cp6=[1.45,3.7]#数字识别点五
cp7=[1.45,1.7]#数字识别点六

cp8=[1.45,0.3]#右降落点
cp9=[-1.7,0.0]#左降落点
cp10=[0.0,0.4]
cp11=[-1.5,2.6]
cp12=[0.0,4.0]

roqu=queue.Queue()
roqu.put(cp0)
roqu.put(cp1)
roqu.put(cp2)
roqu.put(cp4)
# roqu.put(cp9)


font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN#字体类型
imageFlag=True
CE=0.1#坐标误差判断
KNOWN_DISTANCE = 13#飞行高度分米
my_list=''


In [3]:

# 字符串对比
def StrComparison(str1,str2):
    n = len(str1)
    res = []
    for x in str1:
        if x in str2:
            res.append(x)
    #print (n)
    return (n-len(res))
#二维码识别
def QRCode():
    global lst1
    global my_list
    signRight=0
    signLeft=0
    right =False
    left = False
    while True:
        ret,img_QR=cap.read()
        gray = cv2.cvtColor(img_QR, cv2.COLOR_BGR2GRAY)
        barcodes = pyzbar.decode(gray)
        for barcode in barcodes:# 循环读取检测到的条形码
            # 绘条形码、二维码多边形轮廓
            points =[]
            for point in barcode.polygon:
                points.append([point[0], point[1]])
            points = np.array(points,dtype=np.int32).reshape(-1,1,2)
            cv2.polylines(img_QR, [points], isClosed=True, color=(0,0,255),thickness=2)
            # 条形码数据为字节对象，所以如果我们想把它画出来
            # 需要先把它转换成字符串
            barcodeData = barcode.data.decode("UTF-8") #先解码成字符串
            lst1 = []
            lst2 = []
            #data_list = barcodeData.split(',')
            for i, num in enumerate(barcodeData.split(',')):
                if i<3:
                    lst1.append(num)
                else:
                    lst2.append(num)
            my_list = lst2[0]            
            # 绘出图像上的条形码数据和类型
            lst1 = list(map(int, lst1))
            cv2.putText(img_QR, barcodeData, (barcode.polygon[0].x, barcode.polygon[0].y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if StrComparison(str('right'),my_list )<=1:
                signRight+=1
                if signRight == 5:
                    right = True
                    print('right')
            if StrComparison( str('left'),my_list)<=1:
                signRight+=1
                if signRight == 5:
                    right = True
                    print(lst1)
                    print('left')

        if left or right:
            break         
        wid.value = cv2.imencode('.jpg',img_QR)[1].tobytes()
    return lst1,my_list
    
#Roi    
def findRoi(frame, thresValue):
    x = 0
    y = 0
    squareSideLength=0
    kernel = np.ones((3,3), np.uint8)
    frame = cv2.GaussianBlur(frame,(5,5),0)
    # gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # ret,ddst=cv2.threshold(gray,130,255,cv2.THRESH_BINARY)
    edges = cv2.Canny(frame,150,200)
    circles = cv2.HoughCircles(edges,cv2.HOUGH_GRADIENT,1,100,
                                param1=50,param2=35,minRadius=20,maxRadius=1000)
    if isinstance(circles,np.ndarray) :
        circles = np.uint16(np.around(circles))
        circleArea2=0
        for Index,i in enumerate(circles[0,:]):
            circleArea1=3.14*i[2]
            if circleArea1>circleArea2:
                circleArea2=circleArea1
                maxAreaIndex=Index
        x = circles[0,maxAreaIndex,0]
        y = circles[0,maxAreaIndex,1]
        squareSideLength=int(circles[0,maxAreaIndex,2]*2/math.sqrt(2))
        cv2.rectangle(frame,(x-int(squareSideLength/2),y-int(squareSideLength/2)),(x+int(squareSideLength/2),y+int(squareSideLength/2)),(255,0,0),3)
        # RoiImg=frame[y-int(squareSideLength/2):y+int(squareSideLength/2),x-int(squareSideLength/2) :x+int(squareSideLength/2) ]
        # RoiImg = cv2.cvtColor(RoiImg, cv2.COLOR_BGR2GRAY)
        # ret, RoiImg = cv2.threshold(RoiImg,thresValue,255,cv2.THRESH_BINARY_INV)
        # RoiImg = cv2.morphologyEx(RoiImg, cv2.MORPH_OPEN, kernel, iterations=1)
        # RoiImg=cv2.dilate(RoiImg,kernel,iterations=1)
    return x, y,squareSideLength
    
# #寻找数字
# def findDigit(cnn, RoiImg):
#     th = cv2.resize(RoiImg,(28,28),interpolation=cv2.INTER_AREA)
#     newing=[[th]]
#     newing = torch.Tensor(np.array(newing)).float()/255. 
#     predicions=cnn(newing)
#     retult=torch.argmax(predicions).detach().numpy()
#     return retult,th      
# def mnist():
#     global digital
#     global digit
#     print("mnist")
#     digit_list=[]
#     digital=0
#     while True:
#         ret,frame=cap.read()
#         x,y,squareSideLength= findRoi(frame,thresValue)
#         # digit,th = findDigit(cnn,RoiImg)
#         # cv2.rectangle(frame, (int(x2+(w2-h2)/2),y2), (int(x2+(w2+h2)/2),y2+h2), (153,153,0), 2)
#         # cv2.putText(frame, str(digit), (x2,y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2)
#         digit=int(digit)
#         count_dict = {}
#         digit_list.append(digit)
#         # len(digit_list)
#         # print(len(digit_list))
#         if len(digit_list)>=10:
#             for i, num in enumerate(digit_list):
#                 if num in count_dict:
#                     count_dict[num]['count'] += 1
#                     count_dict[num]['index'].append(i)
#                 else:
#             # 如果不存在，则将元素添加到字典中，并将其次数和索引初始化为1
#                     count_dict[num] = {'count': 1, 'index': [i]}
#             # 遍历字典，并输出每个元素的次数和索引
#             for num, count_index in count_dict.items():
#                 most_common_num = max(count_dict.items(), key=lambda x: x[1]['count'])[0]
#                 #print(f"出现次数最多的数字是：{most_common_num}")
#             digital=int(most_common_num)
#             print('digital',digital)
#             break    
#         newEdges = cv2.cvtColor(ddst, cv2.COLOR_GRAY2BGR)
#         newFrame = np.hstack((frame,newEdges))
#         wid.value = cv2.imencode('.jpg',newFrame)[1].tobytes()        
#     return digital

#投放点校准
def massdrop(thresValue):
    global tarx
    global tary
    global routeNodeIndex
    conter = 0
    sum_x=0
    sum_y=0
    while True: 
        ret, frame=cap.read()
        x,y,squareSideLength=findRoi(frame,thresValue)
        if x!=0: 
            width=((x-(frame.shape[1]/2)) * KNOWN_DISTANCE)/ 520
            high=((y-(frame.shape[2]/2)) * KNOWN_DISTANCE)/ 520
            # tarx=tarx+width
            cv2.circle(frame,(x,y),6,(255,255,0),-1)
            cv2.putText(frame, "%.2fft" % width,(frame.shape[1] - 200, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,2.0, (0, 255, 0), 3)
            sum_x+=width/10
            sum_y+=high/10
            conter+=1
        # newEdges = cv2.cvtColor(ddst, cv2.COLOR_GRAY2BGR)
        # newFrame = np.hstack((frame,newEdges))  
        wid.value = cv2.imencode('.jpg',frame)[1].tobytes()
        if conter==10:
            tarx=tarx-sum_x/10
            tary=tary-sum_y/10
            routeNodeIndex+=1
            print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex,tarx,tary))
            break
#路径点更新图像识别线程
def Router():
    global timer
    global routeNodeIndex
    global SendTargetPos
    global CopterLanding
    global LaserArray
    global LaserDistance
    global FlightMode
    global CopterTakingOff
    global routeStartFlag
    global tarx
    global tary
    global routerFlag
    global dataBuf
    global Downflag
    global Continue
    global upnow
    global digital
    global lst1
    global my_list
    global highok
    massdropflag=True
    oddflag=0
    Downflag=0x00
    highok=0x00
    Continue=True
    lst1=[]
    upnow=False
    Downnow=0
    while True:
        SendTargetPos = 1
        if math.sqrt((tarx-pos_Y)**2+(tary+pos_X)**2)<CE and routeStartFlag and routerFlag:
            if pos_X==0 and pos_Y==0 and pos_Z==0:
                        print('t265error')
            if imageFlag:
            #数字识别启动判断    
                # if math.sqrt((cp1[0]-pos_Y)**2+(cp1[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                # if math.sqrt((cp2[0]-pos_Y)**2+(cp2[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                # if math.sqrt((cp3[0]-pos_Y)**2+(cp3[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                # if math.sqrt((cp5[0]-pos_Y)**2+(cp5[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                # if math.sqrt((cp6[0]-pos_Y)**2+(cp6[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                # if math.sqrt((cp7[0]-pos_Y)**2+(cp7[1]+pos_X)**2)<CE: 
                #     digital=mnist()
                    
                    
                #二维码识别启动判断
                if math.sqrt((cp4[0]-pos_Y)**2+(cp4[1]+pos_X)**2)<CE: 
                    #二维码识别   
                    lst1,my_list=QRCode()
                    # oddflag=1
                    # print('oddflag',oddflag)
                if StrComparison(str('left'),my_list )<=1:
                    if len(lst1)!=0:
                        roqu.put(cp3)
                        roqu.put(cp2)
                        roqu.put(cp1)
                        if math.sqrt((cp3[0]-pos_Y)**2+(cp3[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            # massdrop(thresValue)
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            # print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                        if math.sqrt((cp2[0]-pos_Y)**2+(cp2[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            # massdrop(thresValue)
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            # print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                        if math.sqrt((cp1[0]-pos_Y)**2+(cp1[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            # massdrop(thresValue)
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            # print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                    if len(lst1)==0:
                        roqu.put(cp9)
                        if math.sqrt((cp9[0]-pos_Y)**2+(cp9[1]+pos_X)**2)<CE:
                            Downnow=1
                            print('Downnow',Downnow)
                            
                            
                if StrComparison(str('right'),my_list )<=1:
                    if len(lst1)!=0:
                        roqu.put(cp5)
                        roqu.put(cp6)
                        roqu.put(cp7)
                        if math.sqrt((cp5[0]-pos_Y)**2+(cp5[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                        if math.sqrt((cp6[0]-pos_Y)**2+(cp6[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                        if math.sqrt((cp7[0]-pos_Y)**2+(cp7[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
                            Downflag=0x01
                            highok=0x01
                            Continue=False
                            upnow=True 
                            print('Downflag',Downflag)
                            lst1=lst1[1:]
                            print('lst1',len(lst1))
                            if len(lst1)==0:
                                roqu.queue.clear()
                    if len(lst1)==0:
                        roqu.put(cp8)
                        if math.sqrt((cp8[0]-pos_Y)**2+(cp8[1]+pos_X)**2)<CE:
                            Downnow=1
                            print('Downnow',Downnow)                
                            
                            
            # if math.sqrt((cp1[0]-pos_Y)**2+(cp1[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
            #     Downflag=0x01
            #     highok=0x01
            #     Continue=False
            #     upnow=True 
            #     print('highok',highok)
            #     print('Downflag',Downflag)
            # if math.sqrt((cp2[0]-pos_Y)**2+(cp2[1]+pos_X)**2)<CE and Downflag==0x00 and upnow==False:
            #     Downflag=0x01
            #     highok=0x01
            #     Continue=False
            #     upnow=True 
            #     print('highok',highok)
            #     print('Downflag',Downflag)
            if not roqu.empty():
                if Downflag==0x00 and Continue:
                    TargetPosition=roqu.get()
                    routeNodeIndex+=1
                    tarx=float(TargetPosition[0])
                    tary=float(TargetPosition[1])
                    print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex,tarx,tary))
                    upnow=False
                    time.sleep(1)

       
            if Downnow==1:
            # else:
                # pipe.stop()
                #2SendTargetPos = 0             
                if massdropflag:
                    massdrop(thresValue)
                    massdropflag=False
                CopterTakingOff = 1
                if math.sqrt((tarx-pos_Y)**2+(tary+pos_X)**2)<CE:
                    CopterLanding = 1
                    routeNodeIndex= 1
                    closeRouter = True
                    print ("Landing")    
                    routerFlag=False

        
#串口通信线程
def PortCom(port):
    global pipe
    global cfg
    global SendTargetPos
    global CopterLanding
    global CopterTakingOff
    global _265Ready
    global GetOnceCmd
    global routeNodeIndex
    global routeStartFlag
    global closeRouter
    global Downflag
    global Continue
    global highok
    closeRouter = False
    while(True):

        response = port.readline()
        if(response !=None):
            port.flushInput()        
            CmdStr1 = str(b'Start265\n')
            CmdStr2 = str(b'Departures\n')
            CmdStr3 = str(b'Refresh265\n')
            CmdStr4 = str(b'put_in_ok\n')
            CmdStr5 = str(b'high_ok\n')
            CMD = str(response)            
            #刷新265Start265
            if( ( StrComparison( CMD ,CmdStr1 )<=1)  and  GetOnceCmd ==False):
                print(StrComparison( CMD ,CmdStr1 ),response,CMD)
                # Declare RealSense pipeline, encapsulating the actual device and sensors
                pipe = rs.pipeline()
                cfg = rs.config()
                cfg.enable_stream(rs.stream.pose,rs.format.any,framerate=200)
                # Start streaming with requested config
                pipe.start(cfg)
                dd.initData()
                SendTargetPos = 0
                CopterLanding = 0
                _265Ready=True
                GetOnceCmd =True 
                routeStartFlag = True

            elif( ( StrComparison(CMD ,CmdStr2 )<=1)  and  CopterTakingOff == 1 ):
                print(StrComparison(CMD ,CmdStr2 ),response,CMD)
                print("Get!")
                router = Thread(target=Router, args=())
                router.start()
                CopterTakingOff = 0
            elif( StrComparison(CMD ,CmdStr3 )<=1):
                _265Ready = False
                GetOnceCmd = False
                routeNodeIndex= 1
                CopterTakingOff =1
                routeStartFlag = False
                print("ReStart!")
                print( StrComparison(CMD ,CmdStr3 ),response,CMD)
                try:
                    pipe.stop()
                    time.sleep(1.0)               
                except:
                    print("Error2")
            elif( StrComparison(CMD ,CmdStr4 )<=1):
                print(StrComparison(CMD ,CmdStr4 ),response,CMD)
                print("OK!")
                # upnow=True           
                Downflag=0x00
                Continue = True
                print('Downflag',Downflag)
                print('Continue',Continue)
            elif( StrComparison(CMD ,CmdStr5 )<=1):
                # print(StrComparison(CMD ,CmdStr5 ),response,CMD)
                # print("high ok!")
                highok=0x00
                # print('highok',highok)
            
                
            response = 0
            CMD = 0
       
                        
        time.sleep(0.02) 

if __name__ == '__main__':
    global routeNodeIndex
    global SendTargetPos
    global CopterLanding
    global LaserArray
    global LaserDistance
    global FlightMode
    global pipe
    global _265Ready
    global GetOnceCmd
    global CheckSum
    global tarx
    global tary
    global routerFlag
    global dataBuf
    global Downflag
    global highok
    port = serial.Serial(port="/dev/ttyAMA0",baudrate=230400,stopbits=1,parity=serial.PARITY_NONE,timeout=1000)
    kernel = np.ones((4, 4), np.uint8) #膨胀算子
    thresValue=50                      #阈值
    wid = ipywidgets.Image() 
    #打开摄像头  
    cap=cv2.VideoCapture(0)
    if not cap.isOpened():
        cap=cv2.VideoCapture(1)
    if not cap.isOpened():
        raise IOError('Can not open video')
    #设置摄像头参数    
    cap.set(cv2.CAP_PROP_FPS, 30)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,cap_x)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,cap_y)    
    ret,img = cap.read()    
       
    #串口通信线程
    thread_Serial = Thread(target=PortCom, args=(port,))
    thread_Serial.start()
    
    #导入
    TargetPosition=roqu.get()
    tarx=TargetPosition[0]
    tary=TargetPosition[1]
    
    routeNodeIndex = 1
    _265Ready =False
    GetOnceCmd =False
    CheckSum =0 
    dataBuf = [0]*65
    LaserArray =0
    LaserDistance=0.0
    FlightMode =0
    routerFlag =True
    Downflag=0x00
    highok=0x00
    display(wid)  
    try:
        while(True):
            # ret,Frame = cap.read()
            if _265Ready:
                # Wait for the next set of frames from the camera
                frames = pipe.wait_for_frames()
                # Fetch pose frame
                pose = frames.get_pose_frame()
                if pose:   
                    # Print some of the pose data to the terminal
                    data = pose.get_pose_data()
                    dataBuf,pos_X,pos_Y,pos_Z,Euler=dd.solveData(data)
                    if(SendTargetPos == 1):           
                        # posZ = TargetPosition[2]
                        posZ=0.0
                        dataBuf[31] = Downflag
                        dataBuf[32] = highok
                        #dataBuf[43] = 0x20
                        posX_buf = struct.pack("f",tarx) 
                        dataBuf[44] = posX_buf[0]
                        dataBuf[45] = posX_buf[1]
                        dataBuf[46] = posX_buf[2]
                        dataBuf[47] = posX_buf[3] 
                        posY_buf = struct.pack("f",tary)
                        dataBuf[48] = posY_buf[0]
                        dataBuf[49] = posY_buf[1]
                        dataBuf[50] = posY_buf[2]
                        dataBuf[51] = posY_buf[3]
                        posZ_buf = struct.pack("f",posZ) 
                        dataBuf[52] = posZ_buf[0]
                        dataBuf[53] = posZ_buf[1]
                        dataBuf[54] = posZ_buf[2]
                        dataBuf[55] = posZ_buf[3]

                        dataBuf[56] = LaserArray              
                        Laser_Dis = struct.pack("f",LaserDistance)
                        dataBuf[57] = Laser_Dis[0]
                        dataBuf[58] = Laser_Dis[1]
                        dataBuf[59] = Laser_Dis[2]             
                        dataBuf[60] = Laser_Dis[3]                  
                        dataBuf[61] = FlightMode
                   
                    if CopterLanding==1:
                        dataBuf[62] = 0xA5
                    else:
                        dataBuf[62] = 0x00
                    
                    for i in range(0,62):                     
                        CheckSum =  CheckSum + dataBuf[i]
                
                    dataBuf[63] = 0xAA
                    dataBuf[64] = CheckSum & 0x00ff
                    
                    print("\rrpy_rad[0]:{:.2f},rpy_rad[1]:{:.2f},rpy_rad[2]:{:.2f} ,X:{:.2f},Y:{:.2f},Z:{:.2f} ".format(Euler[0]*57.3,Euler[1]*57.3,Euler[2]*57.3,pos_Y,-pos_X,pos_Z),end="")
                    port.write(dataBuf)
                    CheckSum = 0
#                     pipe.stop()
            else:
                dataBuf[0] = 0x55
                dataBuf[1] = 0xAA
                dataBuf[2] = 0xFF
                dataBuf[63] = 0xAA
                dataBuf[64] = 0x00
                port.write(dataBuf)
                time.sleep(0.1)
            # wid.value = cv2.imencode('.jpg',Frame)[1].tobytes()
    finally:
        
        pipe.stop()
        dataBuf[62] = 0xA5
        port.write(dataBuf)
        print("some erro")
#         pipe.stop()

Image(value=b'')

0 b'Start265\n' b'Start265\n'
rpy_rad[0]:-49.51,rpy_rad[1]:-1.19,rpy_rad[2]:0.57 ,X:0.11,Y:0.15,Z:1.15 0 b'Departures\n' b'Departures\n'
Get!
rpy_rad[0]:-48.63,rpy_rad[1]:-1.52,rpy_rad[2]:-0.80 ,X:0.01,Y:0.10,Z:1.23 route node 2: x : -1.7 , y : 1.6  
rpy_rad[0]:-49.15,rpy_rad[1]:-1.25,rpy_rad[2]:1.45 ,X:-1.61,Y:1.58,Z:1.18 route node 3: x : -1.7 , y : 3.5  
rpy_rad[0]:-48.87,rpy_rad[1]:-1.02,rpy_rad[2]:0.57 ,X:-1.61,Y:3.54,Z:1.15 route node 4: x : -0.2 , y : 4.9  
rpy_rad[0]:-48.99,rpy_rad[1]:-0.55,rpy_rad[2]:2.92 ,X:-0.15,Y:4.86,Z:1.11 [1, 2, 6]
left
route node 5: x : -1.7 , y : 5.5  
rpy_rad[0]:-48.51,rpy_rad[1]:-1.51,rpy_rad[2]:2.68 ,X:-1.61,Y:5.52,Z:1.07 2st1
rpy_rad[0]:-48.50,rpy_rad[1]:-1.24,rpy_rad[2]:5.26 ,X:-1.79,Y:5.36,Z:0.33 0 b'put_in_ok\n' b'put_in_ok\n'
OK!
Downflag 0
Continue True
rpy_rad[0]:-47.80,rpy_rad[1]:-1.55,rpy_rad[2]:4.78 ,X:-1.77,Y:5.43,Z:0.55 route node 6: x : -1.7 , y : 3.5  
rpy_rad[0]:-48.49,rpy_rad[1]:-0.73,rpy_rad[2]:4.65 ,X:-1.72,Y:3.60,Z:1.13 lst1 1
rpy

KeyboardInterrupt: 